# QA files for regression prepration

In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

from collections import Counter

import re
import string 

import nltk 
nltk.download('twitter_samples')
from nltk.corpus import twitter_samples
from nltk.corpus import stopwords          # module for stop words that come with NLTK
nltk.download('stopwords')
from nltk.stem import PorterStemmer        # module for stemming
from nltk.tokenize import TweetTokenizer   # module for tokenizing strings

# ignore warning
import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.filterwarnings('ignore')

# for stock price
import yfinance as yf
from yahoofinancials import YahooFinancials
pd.set_option('display.max_columns', None)


[nltk_data] Downloading package twitter_samples to
[nltk_data]     /Users/timliu/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/timliu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# read the csv file 
topic_modelling_df_QA = pd.read_csv('./main_df_output/QA27_topic_modelling_df.csv')
topic_modelling_df_QA

,participants,file_name,date,idx,company_paticipants_yes,other_paticipants_yes,company_name,paraghrph,paraghrph_clean,paraghrph_noun,token,word_count,char_count,sentence_count,avg_word_length,avg_sentence_length,token_len,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24,topic_25,topic_26,topic_27
0,05HFRJ-E Denis Kessler,20190220_SCOR_SE-_Earnings_Call_2019-2-20_FS00...,2019-02-20,559.0,1,0,SCOR SE-,"Maybe before Ian takes the floor, we feel good...",Maybe before takes the floor we feel good and...,floor advertising campaign company board yeste...,"['floor', 'advertising', 'campaign', 'board', ...",33,230,17,6.969697,1.941176,14,0.243455,0.005651,0.011600,0.007116,0.062483,0.003960,0.005926,0.005970,0.006187,0.008405,0.006458,0.307279,0.007090,0.004276,0.008650,0.004288,0.005825,0.129604,0.007003,0.003106,0.006626,0.004905,0.000734,0.005544,0.064401,0.005744,0.067716
1,Aaron Henry Meder,20180620_LEGAL_-_GEN_GRP-_Shareholder_Mtg_Call...,2018-06-20,236.0,1,0,LEGAL -,16683830 Yeah. It probably depends a little ...,Yeah It probably depends a little bit on ki...,bit duration type mandate lot customization ma...,"['duration', 'mandate', 'customization', 'land...",34,226,15,6.647059,2.266667,15,0.116736,0.005332,0.010945,0.006714,0.002470,0.003737,0.005591,0.005633,0.062322,0.007931,0.006093,0.007504,0.006689,0.004035,0.008161,0.004046,0.005496,0.404702,0.006607,0.059415,0.006252,0.004628,0.000692,0.005231,0.004280,0.005420,0.233342
2,Abid Hussain,20130516_Aviva_PLC-_Guidance_Call_2013-5-16_SD...,2013-05-16,261.0,0,1,Aviva PLC-,"17127644 Good morning, can you hear me? Good...",Good morning can you hear me? Good morning ...,morning morning question loan thinking loan nu...,"['loan', 'thinking', 'loan', 'option', 'loan',...",17,105,7,6.176471,2.428571,9,0.005701,0.008065,0.016555,0.010156,0.003736,0.005652,0.008457,0.008520,0.008831,0.011996,0.009216,0.096790,0.351876,0.006103,0.012345,0.006119,0.008314,0.014092,0.009994,0.004434,0.009456,0.007000,0.001047,0.007913,0.006474,0.008198,0.352961
3,Abid Hussain,20131107_Aviva_PLC-_Guidance_Call_2013-11-7_SD...,2013-11-07,354.0,0,1,Aviva PLC-,"17127644 Hi, morning, all. Just three questi...",Hi morning all Just three questions if I ca...,hi morning question cost run rate cost save en...,"['save', 'save', 'remittance', 'impd', 'direct...",32,175,15,5.468750,2.133333,6,0.007666,0.010845,0.022261,0.013656,0.005024,0.007600,0.011372,0.011457,0.011874,0.016131,0.012393,0.589696,0.013606,0.008207,0.016600,0.008228,0.011180,0.018949,0.013439,0.005962,0.012716,0.124299,0.001408,0.010640,0.008705,0.011023,0.015066
4,Abid Hussain,20150506_LEGAL_-_GEN_GRP-_Guidance_Call_2015-5...,2015-05-06,299.0,0,1,LEGAL -,"17127644 Three questions from me, if I may. ...",Three questions from me if I may The first ...,question annuity surface case q landscape end ...,"['surface', 'landscape', 'cofund', 'slowdown',...",26,169,9,6.500000,2.888889,9,0.005701,0.008065,0.016555,0.010156,0.003736,0.005652,0.008457,0.008520,0.008831,0.011996,0.009216,0.011351,0.095558,0.006103,0.183223,0.006119,0.008314,0.014092,0.009994,0.260751,0.094896,0.007000,0.001047,0.007913,0.006474,0.008198,0.182083
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30758,Yusuke Yabumoto,20151125_MS-AD_INSURANCE-_Earnings_Call_2015-1...,2015-11-25,365.0,0,1,MS-AD INSURANCE-,"16424475 Yabumoto, Mizuho Securities. Let me...",Mizuho Securities Let me ask two questions...,question share repurchase core profit relation...,"['repurchase', 'core', 'relation', 'core', 'me...",27,168,7,6.222222,3.857143,7,0.006876,0.009727,0.019967,0.012249,0.004506,0.006817,0.010200,0.010276,0.010650,0.014468,0.011116,0.013690,0.012204,0.007361,0.014889,0.007381,0.010028,0.016996,0.630342,0.

# Define Functions

In [4]:
# clean text
def process_text(text):
    stemmer = PorterStemmer()
    stopwords_english = stopwords.words('english')
    #text = text.str
    text = str(text)
    text = re.sub(r'\$\w*', '', text)
    text = re.sub(r'^RT[\s]+', '', text)
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text)
    text = re.sub(r'#', '', text)
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,reduce_len=True)
    text_tokens = tokenizer.tokenize(text)

    text_clean = []
    for word in text_tokens:
        if (word not in stopwords_english and  
                word not in string.punctuation): 
            stem_word = stemmer.stem(word)  # stemming word
            text_clean.append(stem_word)
            
    sentence = ' '.join(text_clean)
    
    return sentence

In [5]:
# calculate the stock price change
def stock_price_change(stock_list, start_date, end_date):
    # function for find price change
    def price_change(aapl_df, n_day):
        before = []
        after = []
        for i in range(len(aapl_df)):
            # price on that date
            prc_tdy = aapl_df['Close'][i]

            if ((i >= n_day) & 
                (i < (len(aapl_df)-n_day))):
                # price change before n_day
                prc_before = aapl_df['Close'][i-n_day]
                prc_change_before = (prc_tdy - prc_before)/prc_before
                # price change after n_day
                prc_after = aapl_df['Close'][i+n_day]
                prc_change_after = (prc_after - prc_tdy)/prc_tdy
            else:
                prc_change_before = 0
                prc_change_after = 0
            # append into list
            before.append(prc_change_before)
            after.append(prc_change_after)
        return before, after
        
    # tickers and its closing stock price
    stock_df = pd.DataFrame()
    for i in stock_list:
        aapl_df = yf.download(i, 
                        start= start_date,  # start='2010-12-01', # start 1 month before
                        end = end_date,   # end='2022-01-30', # end 1 month later
                        progress=False,)
        aapl_df = aapl_df.reset_index(drop=False)
        # add ticker name
        aapl_df['ticker']=i
        # add price change
        # aapl_df['D0'] = aapl_df['Close']
        aapl_df['D-1'],aapl_df['D+1'] = price_change(aapl_df, 1)
        aapl_df['D-2'],aapl_df['D+2'] = price_change(aapl_df, 2)
        aapl_df['D-3'],aapl_df['D+3'] = price_change(aapl_df, 3)
        aapl_df['D-5'],aapl_df['D+5'] = price_change(aapl_df, 5)
        aapl_df['D-10'],aapl_df['D+10'] = price_change(aapl_df, 10)
        aapl_df['D-15'],aapl_df['D+15'] = price_change(aapl_df, 15)
        # append into one dataframe 
        stock_df = stock_df.append(aapl_df)
        
    # drop redundancy columns
    stock_price_df = stock_df.drop(columns=['Open', 'High','Low','Adj Close','Volume'])
    stock_price_df = stock_price_df.rename(columns={'Date': 'date'})
    # drop duplicated rows
    stock_price_df = stock_price_df.drop_duplicates()
    return stock_price_df

# Clean text and splitting to sentence

In [6]:
df = topic_modelling_df_QA.drop(['participants','idx','company_paticipants_yes','other_paticipants_yes',
                'paraghrph_noun', 'word_count', 'char_count', 'sentence_count',
                'avg_word_length', 'avg_sentence_length'], axis = 1)
df = df.rename(columns = {'paraghrph':'paraghraph','paraghrph_clean':'paraghraph_clean'})
df = df.sort_values(by=['file_name'])
df = df.reset_index(drop=True)

# splotting into the sentences
df['paraghraph'] = df['paraghraph'].apply(lambda r: r.replace("Mr. ","Mr."))
df['sentence'] = df['paraghraph'].apply(lambda r: r.split(". "))
df = df.explode('sentence')

# Apply the function to clean the text
df['clean_text'] = df['sentence'].apply(lambda r: process_text(r))
df.head(5)

df.head(5)

,file_name,date,company_name,paraghraph,paraghraph_clean,token,token_len,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24,topic_25,topic_26,topic_27,sentence,clean_text
0,20110202 _Mapfre_SA_Earnings_Call_SD0000000027...,2011-02-02,Mapfre SA,"7573856 Yes, fair enough. And a final questi...",Yes fair enough And a final question which ...,"['banking', 'certainty', 'scenario', 'consolid...",6,0.007666,0.010845,0.022261,0.128543,0.005024,0.007600,0.011372,0.011457,0.011874,0.016131,0.012393,0.015263,0.243380,0.008207,0.131486,0.008228,0.01118,0.018949,0.013439,0.005962,0.012716,0.009413,0.001408,0.01064,0.008705,0.011023,0.244839,"7573856 Yes, fair enough",7573856 ye fair enough
0,20110202 _Mapfre_SA_Earnings_Call_SD0000000027...,2011-02-02,Mapfre SA,"7573856 Yes, fair enough. And a final questi...",Yes fair enough And a final question which ...,"['banking', 'certainty', 'scenario', 'consolid...",6,0.007666,0.010845,0.022261,0.128543,0.005024,0.007600,0.011372,0.011457,0.011874,0.016131,0.012393,0.015263,0.243380,0.008207,0.131486,0.008228,0.01118,0.018949,0.013439,0.005962,0.012716,0.009413,0.001408,0.01064,0.008705,0.011023,0.244839,"And a final question, which is more strategic",final question strateg
0,20110202 _Mapfre_SA_Earnings_Call_SD0000000027...,2011-02-02,Mapfre SA,"7573856 Yes, fair enough. And a final questi...",Yes fair enough And a final question which ...,"['banking', 'certainty', 'scenario', 'consolid...",6,0.007666,0.010845,0.022261,0.128543,0.005024,0.007600,0.011372,0.011457,0.011874,0.016131,0.012393,0.015263,0.243380,0.008207,0.131486,0.008228,0.01118,0.018949,0.013439,0.005962,0.012716,0.009413,0.001408,0.01064,0.008705,0.011023,0.244839,I mean clearly there has been quite a lot of c...,mean clearli quit lot chang spanish bank system
0,20110202 _Mapfre_SA_Earnings_Call_SD0000000027...,2011-02-02,Mapfre SA,"7573856 Yes, fair enough. And a final questi...",Yes fair enough And a final question which ...,"['banking', 'certainty', 'scenario', 'consolid...",6,0.007666,0.010845,0.022261,0.128543,0.005024,0.007600,0.011372,0.011457,0.011874,0.016131,0.012393,0.015263,0.243380,0.008207,0.131486,0.008228,0.01118,0.018949,0.013439,0.005962,0.012716,0.009413,0.001408,0.01064,0.008705,0.011023,0.244839,And I was wondering whether for your final div...,wonder whether final dividend posit say 100 ce...
1,20110202 _Mapfre_SA_Earnings_Call_SD0000000027...,2011-02-02,Mapfre SA,3910673 I don't think that the current situa...,I don't think that the current situation at...,"['problem', 'ization', 'agreement', 'clarifica...",10,0.083966,0.322286,0.093966,0.009356,0.003442,0.241349,0.007791,0.007849,0.008135,0.011052,0.008491,0.010457,0.009322,0.005623,0.011373,0.005638,0.00766,0.012983,0.087922,0.004085,0.008712,0.006449,0.000965,0.00729,0.005964,0.007552,0.010322,3910673 I don't think that the current situa...,3910673 think current situat caja madrid take ...


# Weighted topic Probability by files

In [7]:
topic_df = df.copy()
# cal len paragraph
topic_df['len_para'] = topic_df['paraghraph'].apply(lambda r: len(r))

In [8]:
cal_len_docs = topic_df[['file_name','len_para']]
# Use GroupBy() to compute the sum of the document
cal_len_docs = cal_len_docs.groupby('file_name').sum()
cal_len_docs = cal_len_docs.rename(columns={"len_para":"len_docs"})
cal_len_docs = cal_len_docs.reset_index(drop=False)
cal_len_docs.head(5)

,file_name,len_docs
0,20110202 _Mapfre_SA_Earnings_Call_SD0000000027...,165914
1,20110202_LEGAL_-_GEN_GRP-_Guidance_Call_2011-2...,73395
2,20110203_Markel_Corp-_Earnings_Call_2011-2-3_S...,208353
3,20110208_Beazley_PLC-_Earnings_Call_2011-2-8_S...,146531
4,20110209_INTACT_FINANCIAL-_Earnings_Call_2011-...,162450


In [9]:
# calculate weighted average for topic probability
# -----> merge the cal_len_docs
merge_df = cal_len_docs.merge(topic_df, how='inner', on='file_name')
# -----> caculate the weight of the sentiment
merge_df['weighted']=merge_df['len_para']/merge_df['len_docs']


In [10]:
merge_df.head(1)

,file_name,len_docs,date,company_name,paraghraph,paraghraph_clean,token,token_len,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24,topic_25,topic_26,topic_27,sentence,clean_text,len_para,weighted
0,20110202 _Mapfre_SA_Earnings_Call_SD0000000027...,165914,2011-02-02,Mapfre SA,"7573856 Yes, fair enough. And a final questi...",Yes fair enough And a final question which ...,"['banking', 'certainty', 'scenario', 'consolid...",6,0.007666,0.010845,0.022261,0.128543,0.005024,0.0076,0.011372,0.011457,0.011874,0.016131,0.012393,0.015263,0.24338,0.008207,0.131486,0.008228,0.01118,0.018949,0.013439,0.005962,0.012716,0.009413,0.001408,0.01064,0.008705,0.011023,0.244839,"7573856 Yes, fair enough",7573856 ye fair enough,493,0.002971


In [11]:
# -----> give the weighted average of the probability based on the paragraph length
for i in range (1,28):
    merge_df['topic_'+str(i)] = merge_df['topic_'+str(i)]*merge_df['weighted']
# -----> sum WA sentiment of each files
topic_df = merge_df.copy()
topic_df = topic_df.groupby(['file_name','date','company_name']).sum().reset_index(drop=False)
# drop the column: 'len_docs', 'len_para', 'weighted'
topic_df = topic_df.drop(['len_docs','len_para','weighted'], axis = 1)
topic_df

,file_name,date,company_name,token_len,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24,topic_25,topic_26,topic_27
0,20110202 _Mapfre_SA_Earnings_Call_SD0000000027...,2011-02-02,Mapfre SA,1675,0.016291,0.114667,0.047784,0.009255,0.002902,0.039433,0.030736,0.016444,0.110710,0.009319,0.047246,0.010767,0.133934,0.024089,0.022317,0.063570,0.026110,0.026061,0.035479,0.015219,0.045993,0.059995,0.000813,0.040893,0.014855,0.018188,0.016931
1,20110202_LEGAL_-_GEN_GRP-_Guidance_Call_2011-2...,2011-02-02,LEGAL -,675,0.025252,0.024645,0.022416,0.057259,0.003718,0.005625,0.008416,0.014398,0.014728,0.011938,0.081514,0.087484,0.038090,0.006074,0.132904,0.012030,0.008274,0.099452,0.151296,0.013312,0.064402,0.012907,0.001042,0.007875,0.006443,0.045717,0.042790
2,20110203_Markel_Corp-_Earnings_Call_2011-2-3_S...,2011-02-03,Markel Corp-,2445,0.045810,0.014942,0.125422,0.018155,0.006783,0.015115,0.049008,0.029997,0.006608,0.052420,0.037126,0.031761,0.031666,0.031956,0.092358,0.027831,0.049387,0.130988,0.061965,0.003318,0.014127,0.007062,0.000783,0.018789,0.004844,0.051541,0.040236
3,20110208_Beazley_PLC-_Earnings_Call_2011-2-8_S...,2011-02-08,Beazley PLC-,1350,0.007589,0.020448,0.110164,0.057239,0.002750,0.016586,0.054526,0.060280,0.046794,0.015044,0.006785,0.044415,0.011152,0.023616,0.053546,0.004505,0.041066,0.064178,0.099604,0.003264,0.143782,0.036217,0.000771,0.012814,0.004766,0.006035,0.052062
4,20110209_INTACT_FINANCIAL-_Earnings_Call_2011-...,2011-02-09,INTACT FINANCIAL-,2052,0.028421,0.103133,0.025766,0.051370,0.012500,0.009217,0.042936,0.037174,0.015251,0.159217,0.030078,0.031709,0.037059,0.022608,0.027561,0.004797,0.116600,0.055153,0.010557,0.005229,0.022458,0.024550,0.032985,0.023870,0.013094,0.006427,0.050282
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1817,20220310_Sanlam_Ltd-_Earnings_Call_2022-3-10_D...,2022-03-10,Sanlam Ltd-,928,0.252895,0.011434,0.040547,0.007775,0.002860,0.004327,0.006475,0.014936,0.006761,0.039612,0.034929,0.008690,0.047743,0.004673,0.009451,0.010468,0.006365,0.010789,0.152126,0.013913,0.015455,0.011143,0.098356,0.037839,0.098514,0.020213,0.031712
1818,20220323_Poste_Italiane_SpA-_Earnings_Call_202...,2022-03-23,Poste Italiane,2260,0.018158,0.039076,0.039430,0.104581,0.002978,0.133730,0.022963,0.008222,0.059963,0.112432,0.007347,0.117423,0.008066,0.045589,0.044818,0.031885,0.010428,0.025770,0.013818,0.011997,0.011339,0.005580,0.031788,0.015932,0.026801,0.011835,0.038051
1819,20220324_Helvetia_Holding_AG-_Earnings_Call_20...,2022-03-24,Helvetia Holding,4015,0.004065,0.068343,0.044193,0.040539,0.002664,0.015006,0.027517,0.044443,0.044321,0.044512,0.111986,0.125690,0.012040,0.022061,0.075848,0.005382,0.010179,0.026877,0.063206,0.023560,0.013088,0.027195,0.019778,0.021138,0.008976,0.035685,0.061709
1820,20220426_Tryg_A-S-_Earnings_Call_2022-4-26_DN0...,2022-04-26,Tryg A-S-,2868,0.008547,0.006471,0.051720,0.090168,0.200004,0.006431,0.027390,0.008593,0.033769,0.067336,0.015486,0.035696,0.008119,0.007361,0.069322,0.099493,0.018008,0.043478,0.048876,0.013344,0.021733,0.005617,0.030273,0.024702,0.012248,0.020325,0.025490


# Weighted Sentiment Score by files

In [12]:
import pickle
sentiment = pickle.load(open('sentiment_model.pkl', 'rb'))
df['sentiment'] = sentiment.predict(df['clean_text'])
df.head(3)

,file_name,date,company_name,paraghraph,paraghraph_clean,token,token_len,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24,topic_25,topic_26,topic_27,sentence,clean_text,sentiment
0,20110202 _Mapfre_SA_Earnings_Call_SD0000000027...,2011-02-02,Mapfre SA,"7573856 Yes, fair enough. And a final questi...",Yes fair enough And a final question which ...,"['banking', 'certainty', 'scenario', 'consolid...",6,0.007666,0.010845,0.022261,0.128543,0.005024,0.0076,0.011372,0.011457,0.011874,0.016131,0.012393,0.015263,0.24338,0.008207,0.131486,0.008228,0.01118,0.018949,0.013439,0.005962,0.012716,0.009413,0.001408,0.01064,0.008705,0.011023,0.244839,"7573856 Yes, fair enough",7573856 ye fair enough,1
0,20110202 _Mapfre_SA_Earnings_Call_SD0000000027...,2011-02-02,Mapfre SA,"7573856 Yes, fair enough. And a final questi...",Yes fair enough And a final question which ...,"['banking', 'certainty', 'scenario', 'consolid...",6,0.007666,0.010845,0.022261,0.128543,0.005024,0.0076,0.011372,0.011457,0.011874,0.016131,0.012393,0.015263,0.24338,0.008207,0.131486,0.008228,0.01118,0.018949,0.013439,0.005962,0.012716,0.009413,0.001408,0.01064,0.008705,0.011023,0.244839,"And a final question, which is more strategic",final question strateg,1
0,20110202 _Mapfre_SA_Earnings_Call_SD0000000027...,2011-02-02,Mapfre SA,"7573856 Yes, fair enough. And a final questi...",Yes fair enough And a final question which ...,"['banking', 'certainty', 'scenario', 'consolid...",6,0.007666,0.010845,0.022261,0.128543,0.005024,0.0076,0.011372,0.011457,0.011874,0.016131,0.012393,0.015263,0.24338,0.008207,0.131486,0.008228,0.01118,0.018949,0.013439,0.005962,0.012716,0.009413,0.001408,0.01064,0.008705,0.011023,0.244839,I mean clearly there has been quite a lot of c...,mean clearli quit lot chang spanish bank system,-1


In [13]:
# Weight of each sentimentc
# cal len paragraph
df['len_para'] = df['paraghraph'].apply(lambda r: len(r))
# cal len sentence 
df['len_sent'] = df['sentence'].apply(lambda r: len(r))
df.head(3)

,file_name,date,company_name,paraghraph,paraghraph_clean,token,token_len,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24,topic_25,topic_26,topic_27,sentence,clean_text,sentiment,len_para,len_sent
0,20110202 _Mapfre_SA_Earnings_Call_SD0000000027...,2011-02-02,Mapfre SA,"7573856 Yes, fair enough. And a final questi...",Yes fair enough And a final question which ...,"['banking', 'certainty', 'scenario', 'consolid...",6,0.007666,0.010845,0.022261,0.128543,0.005024,0.0076,0.011372,0.011457,0.011874,0.016131,0.012393,0.015263,0.24338,0.008207,0.131486,0.008228,0.01118,0.018949,0.013439,0.005962,0.012716,0.009413,0.001408,0.01064,0.008705,0.011023,0.244839,"7573856 Yes, fair enough",7573856 ye fair enough,1,493,26
0,20110202 _Mapfre_SA_Earnings_Call_SD0000000027...,2011-02-02,Mapfre SA,"7573856 Yes, fair enough. And a final questi...",Yes fair enough And a final question which ...,"['banking', 'certainty', 'scenario', 'consolid...",6,0.007666,0.010845,0.022261,0.128543,0.005024,0.0076,0.011372,0.011457,0.011874,0.016131,0.012393,0.015263,0.24338,0.008207,0.131486,0.008228,0.01118,0.018949,0.013439,0.005962,0.012716,0.009413,0.001408,0.01064,0.008705,0.011023,0.244839,"And a final question, which is more strategic",final question strateg,1,493,45
0,20110202 _Mapfre_SA_Earnings_Call_SD0000000027...,2011-02-02,Mapfre SA,"7573856 Yes, fair enough. And a final questi...",Yes fair enough And a final question which ...,"['banking', 'certainty', 'scenario', 'consolid...",6,0.007666,0.010845,0.022261,0.128543,0.005024,0.0076,0.011372,0.011457,0.011874,0.016131,0.012393,0.015263,0.24338,0.008207,0.131486,0.008228,0.01118,0.018949,0.013439,0.005962,0.012716,0.009413,0.001408,0.01064,0.008705,0.011023,0.244839,I mean clearly there has been quite a lot of c...,mean clearli quit lot chang spanish bank system,-1,493,82


In [14]:
cal_len_docs = df[['file_name','len_para']]
# Use GroupBy() to compute the sum of the document
cal_len_docs = cal_len_docs.groupby('file_name').sum()
cal_len_docs = cal_len_docs.rename(columns={"len_para":"len_docs"})
cal_len_docs = cal_len_docs.reset_index(drop=False)
cal_len_docs.head(5)

,file_name,len_docs
0,20110202 _Mapfre_SA_Earnings_Call_SD0000000027...,165914
1,20110202_LEGAL_-_GEN_GRP-_Guidance_Call_2011-2...,73395
2,20110203_Markel_Corp-_Earnings_Call_2011-2-3_S...,208353
3,20110208_Beazley_PLC-_Earnings_Call_2011-2-8_S...,146531
4,20110209_INTACT_FINANCIAL-_Earnings_Call_2011-...,162450


In [15]:
# calculate weighted average for sentiment
# -----> merge the cal_len_docs
merge_df = cal_len_docs.merge(df, how='inner', on='file_name')
# -----> caculate the weight of the sentiment
merge_df['weighted']=merge_df['len_sent']/merge_df['len_docs']
# -----> give the weighted average of the sentiment score based on the sentence
merge_df['WA_sentiment'] = merge_df['weighted']*merge_df['sentiment']
# -----> sum WA sentiment of each files
sentiment_df = merge_df[['file_name','date','company_name','WA_sentiment']]
sentiment_df = sentiment_df.groupby(['file_name','date','company_name']).sum().reset_index(drop=False)
sentiment_df

,file_name,date,company_name,WA_sentiment
0,20110202 _Mapfre_SA_Earnings_Call_SD0000000027...,2011-02-02,Mapfre SA,0.004315
1,20110202_LEGAL_-_GEN_GRP-_Guidance_Call_2011-2...,2011-02-02,LEGAL -,0.037264
2,20110203_Markel_Corp-_Earnings_Call_2011-2-3_S...,2011-02-03,Markel Corp-,0.025860
3,20110208_Beazley_PLC-_Earnings_Call_2011-2-8_S...,2011-02-08,Beazley PLC-,0.010728
4,20110209_INTACT_FINANCIAL-_Earnings_Call_2011-...,2011-02-09,INTACT FINANCIAL-,0.000277
...,...,...,...,...
1817,20220310_Sanlam_Ltd-_Earnings_Call_2022-3-10_D...,2022-03-10,Sanlam Ltd-,0.019666
1818,20220323_Poste_Italiane_SpA-_Earnings_Call_202...,2022-03-23,Poste Italiane,0.036341
1819,20220324_Helvetia_Holding_AG-_Earnings_Call_20...,2022-03-24,Helvetia Holding,0.018238
1820,20220426_Tryg_A-S-_Earnings_Call_2022-4-26_DN0...,2022-04-26,Tryg A-S-,0.017976


# merge the sentiment_df and topic_df together

In [16]:
# merge the sentiment_df and topic_df together
QA_model_df = topic_df.merge(sentiment_df, how='inner', on=['file_name','date','company_name'])
QA_model_df

,file_name,date,company_name,token_len,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24,topic_25,topic_26,topic_27,WA_sentiment
0,20110202 _Mapfre_SA_Earnings_Call_SD0000000027...,2011-02-02,Mapfre SA,1675,0.016291,0.114667,0.047784,0.009255,0.002902,0.039433,0.030736,0.016444,0.110710,0.009319,0.047246,0.010767,0.133934,0.024089,0.022317,0.063570,0.026110,0.026061,0.035479,0.015219,0.045993,0.059995,0.000813,0.040893,0.014855,0.018188,0.016931,0.004315
1,20110202_LEGAL_-_GEN_GRP-_Guidance_Call_2011-2...,2011-02-02,LEGAL -,675,0.025252,0.024645,0.022416,0.057259,0.003718,0.005625,0.008416,0.014398,0.014728,0.011938,0.081514,0.087484,0.038090,0.006074,0.132904,0.012030,0.008274,0.099452,0.151296,0.013312,0.064402,0.012907,0.001042,0.007875,0.006443,0.045717,0.042790,0.037264
2,20110203_Markel_Corp-_Earnings_Call_2011-2-3_S...,2011-02-03,Markel Corp-,2445,0.045810,0.014942,0.125422,0.018155,0.006783,0.015115,0.049008,0.029997,0.006608,0.052420,0.037126,0.031761,0.031666,0.031956,0.092358,0.027831,0.049387,0.130988,0.061965,0.003318,0.014127,0.007062,0.000783,0.018789,0.004844,0.051541,0.040236,0.025860
3,20110208_Beazley_PLC-_Earnings_Call_2011-2-8_S...,2011-02-08,Beazley PLC-,1350,0.007589,0.020448,0.110164,0.057239,0.002750,0.016586,0.054526,0.060280,0.046794,0.015044,0.006785,0.044415,0.011152,0.023616,0.053546,0.004505,0.041066,0.064178,0.099604,0.003264,0.143782,0.036217,0.000771,0.012814,0.004766,0.006035,0.052062,0.010728
4,20110209_INTACT_FINANCIAL-_Earnings_Call_2011-...,2011-02-09,INTACT FINANCIAL-,2052,0.028421,0.103133,0.025766,0.051370,0.012500,0.009217,0.042936,0.037174,0.015251,0.159217,0.030078,0.031709,0.037059,0.022608,0.027561,0.004797,0.116600,0.055153,0.010557,0.005229,0.022458,0.024550,0.032985,0.023870,0.013094,0.006427,0.050282,0.000277
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1817,20220310_Sanlam_Ltd-_Earnings_Call_2022-3-10_D...,2022-03-10,Sanlam Ltd-,928,0.252895,0.011434,0.040547,0.007775,0.002860,0.004327,0.006475,0.014936,0.006761,0.039612,0.034929,0.008690,0.047743,0.004673,0.009451,0.010468,0.006365,0.010789,0.152126,0.013913,0.015455,0.011143,0.098356,0.037839,0.098514,0.020213,0.031712,0.019666
1818,20220323_Poste_Italiane_SpA-_Earnings_Call_202...,2022-03-23,Poste Italiane,2260,0.018158,0.039076,0.039430,0.104581,0.002978,0.133730,0.022963,0.008222,0.059963,0.112432,0.007347,0.117423,0.008066,0.045589,0.044818,0.031885,0.010428,0.025770,0.013818,0.011997,0.011339,0.005580,0.031788,0.015932,0.026801,0.011835,0.038051,0.036341
1819,20220324_Helvetia_Holding_AG-_Earnings_Call_20...,2022-03-24,Helvetia Holding,4015,0.004065,0.068343,0.044193,0.040539,0.002664,0.015006,0.027517,0.044443,0.044321,0.044512,0.111986,0.125690,0.012040,0.022061,0.075848,0.005382,0.010179,0.026877,0.063206,0.023560,0.013088,0.027195,0.019778,0.021138,0.008976,0.035685,0.061709,0.018238
1820,20220426_Tryg_A-S-_Earnings_Call_2022-4-26_DN0...,2022-04-26,Tryg A-S-,2868,0.008547,0.006471,0.051720,0.090168,0.200004,0.006431,0.027390,0.008593,0.033769,0.067336,0.015486,0.035696,0.008119,0.007361,0.069322,0.099493,0.018008,0.043478,0.048876,0.013344,0.021733,0.005617,0.030273,0.024702,0.012248,0.020325,0.025490,0.017976


# Stock Price

In [17]:
# List all yahoo tickers
yahoo_ticker_list = [
    # Motor/Personal
    'ADM.L','DLG.L','SBRE.L','SAGA.L','AGS.BR',
    # Global Commercial
    'ALV.DE','CS.PA','ZURN.SW','G.MI',
    # London Market 
    'BEZ.L','HSX.L','LRE.L',
    # LN Equity 
    'LLOY.L',
    # US - Specialty/P&C/Reinsurance
    'AIG','AXS','TRV','ACGL','RNR','RE','MKL','HIG','ARGO','BRK-B','CB',
    # European (Re)Insurers
    'SCR.PA','MUV2.DE','SREN.SW','HNR1.DE',
    # Japanese & Pacific
    '8766.T','8630.T','8725.T','QBE.AX',
    # Run-off
    'ESGR','FFH.TO','RQIH.L',
    # Life Groups & Retail Life
    'PRU.L','MNG.L','LGEN.L','AV.L','PHNX.L','QLT.L','JUST.L','STJ.L','AGN.AS','DSY.JO','SLM.JO',
    # Other insurers
    'STB.OL','CNP.PA','GJF.OL','PST.MI','NN.AS','TOP.CO','BALN.SW',
    'SAMPO.HE','MAP.MC','TRYG.CO','0RHS.IL','HELN.SW','IFC.TO'
]

# Match it with company names from BoE
Insurer_Names_df = pd.read_excel('./input/Insurer_Names_for_possible_NLP_analysis.xlsx')  
Insurer_Names = Insurer_Names_df.drop(['Unnamed: 0', 'Unnamed: 4'], axis=1)
Insurer_Names = Insurer_Names.drop([0])
Insurer_Names.rename(columns={"Unnamed: 1": "Company", "Unnamed: 2": "bb ticker", "Unnamed: 3":"Group"}, inplace = True)
Insurer_Names['yahoo ticker'] = yahoo_ticker_list
Insurer_Names.head(5)
# Exported as CSV to manually matched with the extracted file names (column 'company_name' in model_df)
# Insurer_Names[['Company']].to_csv('./main_df_output/company_name_match.csv', index = False)

,Company,bb ticker,Group,yahoo ticker
1,ADMIRAL GROUP,ADM LN,Motor/Personal,ADM.L
2,DIRECT LINE INSU,DLG LN,Motor/Personal,DLG.L
3,SABRE INSUR,SBRE LN,Motor/Personal,SBRE.L
4,SAGA PLC,SAGA LN,Motor/Personal,SAGA.L
5,AGEAS,AGS BB,Motor/Personal,AGS.BR


In [18]:
company_name_match = pd.read_csv('./input/company_name_match.csv')
company_name_match.rename(columns={"Company": "company_name"}, inplace = True)
company_name_match

Groups = {
        "ADMIRAL GROUP" : "Motor/Personal", "DIRECT LINE INSU" : "Motor/Personal", 
        "SABRE INSUR" : "Motor/Personal", "SAGA PLC" : "Motor/Personal",
        "AGEAS" : "Motor/Personal",

        "ALLIANZ SE-REG" : "Global Commercial", "AXA" : "Global Commercial", 
        "ZURICH INSURANCE" : "Global Commercial", "GENERALI ASSIC" : "Global Commercial",

        "BEAZLEY PLC" : "London Market", "HISCOX LTD" : "London Market", 
        "LANCASHIRE HOLDI" : "London Market",
        
        "Society of Lloyd’s" : "LN Equity",

        "AMERICAN INTERNA" : "US - Specialty/P&C/Reinsurance", "AXIS CAPITAL" : "US - Specialty/P&C/Reinsurance",
        "TRAVELERS COS IN" : "US - Specialty/P&C/Reinsurance", "ARCH CAPITAL GRP" : "US - Specialty/P&C/Reinsurance",
        "RENAISSANCERE" : "US - Specialty/P&C/Reinsurance", "EVEREST RE GROUP" : "US - Specialty/P&C/Reinsurance",
        "MARKEL CORP" : "US - Specialty/P&C/Reinsurance", "HARTFORD FINL SV" : "US - Specialty/P&C/Reinsurance",
        "ARGO GROUP INTER" : "US - Specialty/P&C/Reinsurance", "BERKSHIRE HATH-B" : "US - Specialty/P&C/Reinsurance",
        "CHUBB LTD" : "US - Specialty/P&C/Reinsurance",

        "SCOR SE" : "European (Re)Insurers", "MUENCHENER RUE-R" : "European (Re)Insurers", 
        "SWISS RE AG" : "European (Re)Insurers", "HANNOVER RUECK S" : "European (Re)Insurers",

        "TOKIO MARINE HD" : "Japanese & Pacific", "SOMPO HOLDINGS I" : "Japanese & Pacific",
        "MS&AD INSURANCE" : "Japanese & Pacific", "QBE INSURANCE" : "Japanese & Pacific",

        "ENSTAR GROUP LTD" : "Run-off", "FAIRFAX FINL HLD" : "Run-off", "RANDALL & QUILTE" : "Run-off",

        "PRUDENTIAL PLC" : "Life Groups & Retail Life", "M&G PLC" : "Life Groups & Retail Life",
        "LEGAL & GEN GRP" : "Life Groups & Retail Life", "AVIVA PLC" : "Life Groups & Retail Life",
        "PHOENIX GROUP HO" : "Life Groups & Retail Life", "QUILTER PLC" : "Life Groups & Retail Life",
        "JUST GROUP" : "Life Groups & Retail Life", "ST JAMES'S PLACE" : "Life Groups & Retail Life",
        "AEGON NV" : "Life Groups & Retail Life", "DISCOVERY LTD" : "Life Groups & Retail Life",
        "SANLAM LTD" : "Life Groups & Retail Life",

        "STOREBRAND ASA" : "Other insurers", "CNP ASSURANCES" : "Other insurers",
        "GJENSIDIGE FORSI" : "Other insurers", "POSTE ITALIANE" : "Other insurers",
        "NN GROUP" : "Other insurers", "TOPDANMARK A/S" : "Other insurers",
        "BALOISE HOL-REG" : "Other insurers", "SAMPO OYJ-A SHS" : "Other insurers",
        "MAPFRE SA" : "Other insurers", "TRYG A/S" : "Other insurers",
        "ASR NEDERLAND NV" : "Other insurers", "HELVETIA HOL-REG" : "Other insurers",
        "INTACT FINANCIAL" : "Other insurers"
}

yahoo_ticker = {
        "ADMIRAL GROUP" : "ADM.L", "DIRECT LINE INSU" : "DLG.L", 
        "SABRE INSUR" : "SBRE.L", "SAGA PLC" : "SAGA.L",
        "AGEAS" : "AGS.BR", 
        
         "ALLIANZ SE-REG" : "ALV.DE", "AXA" : "CS.PA", 
        "ZURICH INSURANCE" : "ZURN.SW", "GENERALI ASSIC" : "G.MI",

        "BEAZLEY PLC" : "BEZ.L", "HISCOX LTD" : "HSX.L", 
        "LANCASHIRE HOLDI" : "LRE.L",

        "Society of Lloyd’s" : "LLOY.L",

        "AMERICAN INTERNA" : "AIG", "AXIS CAPITAL" : "AXS",
        "TRAVELERS COS IN" : "TRV", "ARCH CAPITAL GRP" : "ACGL",
        "RENAISSANCERE" : "RNR", "EVEREST RE GROUP" : "RE",
        "MARKEL CORP" : "MKL", "HARTFORD FINL SV" : "HIG",
        "ARGO GROUP INTER" : "ARGO", "BERKSHIRE HATH-B" : "BRK-B",
        "CHUBB LTD" : "CB",

        "SCOR SE" : "SCR.PA", "MUENCHENER RUE-R" : "MUV2.DE", 
        "SWISS RE AG" : "SREN.SW", "HANNOVER RUECK S" : "HNR1.DE",

        "TOKIO MARINE HD" : "8766.T", "SOMPO HOLDINGS I" : "8630.T",
        "MS&AD INSURANCE" : "8725.T", "QBE INSURANCE" : "QBE.AX",

        "ENSTAR GROUP LTD" : "ESGR", "FAIRFAX FINL HLD" : "FFH.TO", "RANDALL & QUILTE" : "RQIH.L",

        "PRUDENTIAL PLC" : "PRU.L", "M&G PLC" : "MNG.L",
        "LEGAL & GEN GRP" : "LGEN.L", "AVIVA PLC" : "AV.L",
        "PHOENIX GROUP HO" : "PHNX.L", "QUILTER PLC" : "QLT.L",
        "JUST GROUP" : "JUST.L", "ST JAMES'S PLACE" : "STJ.L",
        "AEGON NV" : "AGN.AS", "DISCOVERY LTD" : "DSY.JO",
        "SANLAM LTD" : "SLM.JO",

        "STOREBRAND ASA" : "STB.OL", "CNP ASSURANCES" : "CNP.PA",
        "GJENSIDIGE FORSI" : "GJF.OL", "POSTE ITALIANE" : "PST.MI",
        "NN GROUP" : "NN.AS", "TOPDANMARK A/S" : "TOP.CO",
        "BALOISE HOL-REG" : "BALN.SW", "SAMPO OYJ-A SHS" : "SAMPO.HE",
        "MAPFRE SA" : "MAP.MC", "TRYG A/S" : "TRYG.CO",
        "ASR NEDERLAND NV" : "0RHS.IL", "HELVETIA HOL-REG" : "HELN.SW",
        "INTACT FINANCIAL" : "IFC.TO"
}

company_name_match['Group'] =company_name_match['Real_Company_Name'].map(Groups)
company_name_match['yfiance_ticker'] =company_name_match['Real_Company_Name'].map(yahoo_ticker)
company_name_match

,Real_Company_Name,company_name,Group,yfiance_ticker
0,ADMIRAL GROUP,Admiral Group,Motor/Personal,ADM.L
1,DIRECT LINE INSU,DIRECT LINE,Motor/Personal,DLG.L
2,SABRE INSUR,SABRE INSUR-,Motor/Personal,SBRE.L
3,SAGA PLC,Saga PLC-,Motor/Personal,SAGA.L
4,AGEAS,Ageas SA-NV-,Motor/Personal,AGS.BR
...,...,...,...,...
62,MAPFRE SA,NaN,Other insurers,MAP.MC
63,TRYG A/S,Tryg A-S-,Other insurers,TRYG.CO
64,ASR NEDERLAND NV,ASR Nederland,Other insurers,0RHS.IL
65,HELVETIA HOL-REG,Helvetia Holding,Other insurers,HELN.SW


In [19]:
df_company = company_name_match[['company_name','Group','yfiance_ticker']]
QA_model_df = df_company.merge(QA_model_df, how='inner', on='company_name')
QA_model_df = QA_model_df.rename(columns={'yfiance_ticker':'ticker'})
QA_model_df

,company_name,Group,ticker,file_name,date,token_len,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24,topic_25,topic_26,topic_27,WA_sentiment
0,Admiral Group,Motor/Personal,ADM.L,20110302_Admiral_Group_PLC-_Earnings_Call_2011...,2011-03-02,973,0.021556,0.008087,0.039312,0.014255,0.086833,0.005668,0.017831,0.016064,0.008855,0.143035,0.054304,0.016883,0.076112,0.022405,0.037905,0.112248,0.024080,0.014131,0.010022,0.022909,0.013243,0.007019,0.086682,0.016135,0.046170,0.016473,0.061784,0.006437
1,Admiral Group,Motor/Personal,ADM.L,20110824_Admiral_Group_PLC-_Earnings_Call_2011...,2011-08-24,3341,0.003300,0.032278,0.045035,0.039645,0.041211,0.044080,0.033313,0.015742,0.010900,0.009654,0.014676,0.019314,0.024082,0.063889,0.035895,0.283111,0.038608,0.081096,0.007970,0.018277,0.009236,0.004052,0.000606,0.018319,0.023183,0.064060,0.018468,0.018945
2,Admiral Group,Motor/Personal,ADM.L,20111109_Admiral_Group_PLC-_Guidance_Call_2011...,2011-11-09,3021,0.011359,0.005525,0.312233,0.034590,0.007595,0.003872,0.060122,0.035827,0.006049,0.008218,0.008899,0.007775,0.021558,0.140446,0.060444,0.028081,0.020184,0.036147,0.025739,0.017081,0.015968,0.014979,0.015555,0.014745,0.055612,0.005616,0.025783,0.010792
3,Admiral Group,Motor/Personal,ADM.L,20120307_Admiral_Group_PLC-_Earnings_Call_2012...,2012-03-07,1764,0.012533,0.020521,0.099658,0.016393,0.118369,0.006756,0.030233,0.030492,0.007297,0.140403,0.031257,0.025380,0.026386,0.045338,0.032607,0.070035,0.030454,0.028821,0.016259,0.017532,0.018432,0.005785,0.035146,0.010052,0.083401,0.016531,0.023929,0.027393
4,Admiral Group,Motor/Personal,ADM.L,20120830_Admiral_Group_PLC-_Earnings_Call_2012...,2012-08-30,1804,0.004212,0.018908,0.037657,0.048443,0.090715,0.008002,0.021395,0.015860,0.006525,0.162249,0.006810,0.022992,0.018967,0.014462,0.015469,0.119229,0.033624,0.089024,0.035749,0.003276,0.026290,0.010584,0.046574,0.016249,0.095245,0.013403,0.018087,-0.008186
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1724,INTACT FINANCIAL-,Other insurers,IFC.TO,20210210_INTACT_FINANCIAL-_Earnings_Call_2021-...,2021-02-10,1673,0.017413,0.073937,0.037053,0.010246,0.002513,0.003802,0.011298,0.021212,0.010613,0.019474,0.010085,0.048132,0.006806,0.022494,0.063724,0.008768,0.166952,0.141144,0.008012,0.011859,0.064679,0.072262,0.031285,0.005322,0.107565,0.011652,0.011702,0.014697
1725,INTACT FINANCIAL-,Other insurers,IFC.TO,20210512_INTACT_FINANCIAL-_Earnings_Call_2021-...,2021-05-12,2331,0.026298,0.067316,0.241967,0.009460,0.028878,0.022940,0.017284,0.007685,0.030012,0.090389,0.055028,0.019481,0.008924,0.011500,0.027739,0.040543,0.088555,0.088775,0.018466,0.003409,0.019674,0.005382,0.027067,0.006083,0.011785,0.011261,0.014099,0.020046
1726,INTACT FINANCIAL-,Other insurers,IFC.TO,20210728_INTACT_FINANCIAL-_Earnings_Call_2021-...,2021-07-28,3005,0.021150,0.100949,0.069813,0.017444,0.006516,0.024776,0.020743,0.054153,0.005831,0.011647,0.009763,0.022648,0.008994,0.016346,0.126584,0.062083,0.138836,0.050676,0.013158,0.002927,0.046821,0.024374,0.020335,0.024062,0.040181,0.016969,0.042220,0.027011
1727,INTACT FINANCIAL-,Other insurers,IFC.TO,20211110_INTACT_FINANCIAL-_Earnings_Call_2021-...,2021-11-10,7025,0.017141,0.133018,0.072820,0.011774,0.031938,0.036239,0.008931,0.012773,0.007990,0.076840,0.014800,0.024016,0.009346,0.008760,0.030084,0.040135,0.263365,0.066674,0.008971,0.022028,0.012723,0.026830,0.010831,0.007291,0.011957,0.005466,0.027259,-0.000520


In [20]:
start_date='2010-12-01'
end_date='2022-01-30'
stock_list = company_name_match['yfiance_ticker']
df_stock = stock_price_change(stock_list, start_date, end_date)

# limit the dat till 2021-12-31
df_stock = df_stock[df_stock['date']<='2021-12-31']

df_stock


1 Failed download:
- CNP.PA: No data found, symbol may be delisted


,date,Close,ticker,D-1,D+1,D-2,D+2,D-3,D+3,D-5,D+5,D-10,D+10,D-15,D+15
0,2010-12-01,1561.000000,ADM.L,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,2010-12-02,1589.000000,ADM.L,0.017937,0.010699,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,2010-12-03,1606.000000,ADM.L,0.010699,0.002491,0.028828,0.005604,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,2010-12-06,1610.000000,ADM.L,0.002491,0.003106,0.013216,-0.021739,0.031390,-0.029193,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,2010-12-07,1615.000000,ADM.L,0.003106,-0.024768,0.005604,-0.032198,0.016362,-0.036533,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2776,2021-12-23,163.240005,IFC.TO,-0.001101,0.001470,0.001841,0.008025,0.012090,0.006493,0.005049,0.002940,0.012153,0.010475,0.004678,-0.010843
2777,2021-12-24,163.479996,IFC.TO,0.001470,0.006545,0.000367,0.005016,0.003314,0.005750,0.003684,0.000856,0.013138,0.004037,0.006774,-0.013335
2778,2021-12-29,164.550003,IFC.TO,0.006545,-0.001519,0.008025,-0.000790,0.006915,-0.005044,0.020212,0.011972,0.020022,-0.002492,0.013114,-0.005409
2779,2021-12-30,164.300003,IFC.TO,-0.001519,0.000730,0.005016,-0.003530,0.006493,-0.004139,0.008347,0.003956,0.033984,-0.022581,0.004586,0.000000


In [21]:
df_stock['date'] = df_stock['date'].astype(str)
# merged company tickers into big dataframe
QA_model_df_final = QA_model_df.merge(df_stock, how='inner', on=['ticker','date'])

# QA_model_df = QA_model_df.join(df_stock.set_index(["date","ticker"]), 
#                          on=["date","ticker"],
#                          how='left'
#                         )
QA_model_df_final = QA_model_df_final.dropna()
QA_model_df_final = QA_model_df_final.reset_index(drop=True)
QA_model_df_final

,company_name,Group,ticker,file_name,date,token_len,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24,topic_25,topic_26,topic_27,WA_sentiment,Close,D-1,D+1,D-2,D+2,D-3,D+3,D-5,D+5,D-10,D+10,D-15,D+15
0,Admiral Group,Motor/Personal,ADM.L,20110302_Admiral_Group_PLC-_Earnings_Call_2011...,2011-03-02,973,0.021556,0.008087,0.039312,0.014255,0.086833,0.005668,0.017831,0.016064,0.008855,0.143035,0.054304,0.016883,0.076112,0.022405,0.037905,0.112248,0.024080,0.014131,0.010022,0.022909,0.013243,0.007019,0.086682,0.016135,0.046170,0.016473,0.061784,0.006437,1655.000000,-0.031030,0.013897,-0.020130,0.015106,-0.003612,-0.000604,-0.011350,-0.003021,-0.029326,-0.060423,-0.030463,-0.045317
1,Admiral Group,Motor/Personal,ADM.L,20110824_Admiral_Group_PLC-_Earnings_Call_2011...,2011-08-24,3341,0.003300,0.032278,0.045035,0.039645,0.041211,0.044080,0.033313,0.015742,0.010900,0.009654,0.014676,0.019314,0.024082,0.063889,0.035895,0.283111,0.038608,0.081096,0.007970,0.018277,0.009236,0.004052,0.000606,0.018319,0.023183,0.064060,0.018468,0.018945,1353.000000,-0.118567,-0.054693,-0.110454,-0.025868,-0.094983,-0.005913,-0.125969,0.011086,-0.031496,0.008130,-0.117417,-0.028825
2,Admiral Group,Motor/Personal,ADM.L,20111109_Admiral_Group_PLC-_Guidance_Call_2011...,2011-11-09,3021,0.011359,0.005525,0.312233,0.034590,0.007595,0.003872,0.060122,0.035827,0.006049,0.008218,0.008899,0.007775,0.021558,0.140446,0.060444,0.028081,0.020184,0.036147,0.025739,0.017081,0.015968,0.014979,0.015555,0.014745,0.055612,0.005616,0.025783,0.010792,887.500000,-0.256077,-0.076056,-0.247881,-0.053521,-0.251686,-0.061972,-0.234914,-0.098028,-0.276691,-0.025352,-0.275510,0.039437
3,Admiral Group,Motor/Personal,ADM.L,20120307_Admiral_Group_PLC-_Earnings_Call_2012...,2012-03-07,1764,0.012533,0.020521,0.099658,0.016393,0.118369,0.006756,0.030233,0.030492,0.007297,0.140403,0.031257,0.025380,0.026386,0.045338,0.032607,0.070035,0.030454,0.028821,0.016259,0.017532,0.018432,0.005785,0.035146,0.010052,0.083401,0.016531,0.023929,0.027393,1144.000000,0.100000,0.015734,0.095785,0.012238,0.080264,0.022727,0.062210,0.036713,0.103182,0.020979,0.179990,0.014860
4,Admiral Group,Motor/Personal,ADM.L,20120830_Admiral_Group_PLC-_Earnings_Call_2012...,2012-08-30,1804,0.004212,0.018908,0.037657,0.048443,0.090715,0.008002,0.021395,0.015860,0.006525,0.162249,0.006810,0.022992,0.018967,0.014462,0.015469,0.119229,0.033624,0.089024,0.035749,0.003276,0.026290,0.010584,0.046574,0.016249,0.095245,0.013403,0.018087,-0.008186,1162.000000,-0.028428,0.020654,-0.012744,-0.010327,-0.022708,-0.044750,-0.020236,-0.023236,-0.016920,-0.059380,-0.005137,-0.067986
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1662,INTACT FINANCIAL-,Other insurers,IFC.TO,20201104_INTACT_FINANCIAL-_Earnings_Call_2020-...,2020-11-04,5291,0.007351,0.116419,0.070342,0.014008,0.005241,0.023266,0.010281,0.046636,0.023587,0.010087,0.088353,0.019017,0.020470,0.016282,0.009608,0.032781,0.070423,0.112735,0.014302,0.002143,0.042310,0.028342,0.013568,0.004315,0.012666,0.134581,0.050887,0.008693,147.750000,0.063332,-0.018274,0.069258,-0.019425,0.073609,-0.060778,0.063179,-0.080203,0.031990,0.000203,0.026255,-0.019628
1663,INTACT FINANCIAL-,Other insurers,IFC.TO,20210210_INTACT_FINANCIAL-_Earnings_Call_2021-...,2021-02-10,1673,0.017413,0.073937,0.037053,0.010246,0.002513,0.003802,0.011298,0.021212,0.010613,0.019474,0.010085,0.048132,0.006806,0.022494,0.063724,0.008768,0.166952,0.141144,0.008012,0.011859,0.064679,0.072262,0.031285,0.005322,0.107565,0.011652,0.011702,0.014697,151.820007,0.034267,-0.013371,0.041432,-0.016994,0.039294,-0.032275,0.050876,-0.041101,0.061678,-0.054802,0.054013,-0.033263
1664,INTACT FINANCIAL-,Other insurers,IFC.TO,20210512_INTACT

In [22]:
QA_model_df_final.to_csv("./regression_df_input/QA_model_df.csv", index = False)